## Poetry Generation v2
In this project, I'll use new method for getting batches and try to get a better result.

### Get the Data
I will use full poetry dataset this point, not only the five words poetries. Poetries are seperated by '\n'

In [1]:
import helper

data_dir = './data/poetry_data.txt'

text = helper.load_data(data_dir)

## Explore the Data

In [2]:
import numpy as np

view_poetry_range = (100, 110)

print('Dataset Stats')
print('Roughly the number of unique Chinese Character: {}'.format(len({word: None for word in list(text)})))

poetries = text.split('\n')
print('Number of poetries: {}'.format(len(poetries)))

charactors_count = [len(list(poetry)) for poetry in poetries]
print("Averate number of charactors in each poetry: {}".format(np.average(charactors_count)))
print("Max number of charactors in poetry: {}".format(np.max(charactors_count)))
print()
print('The sentences {} to {}:'.format(*view_poetry_range))
print('\n'.join(poetries[view_poetry_range[0]:view_poetry_range[1]]))

Dataset Stats
Roughly the number of unique Chinese Character: 7077
Number of poetries: 41134
Averate number of charactors in each poetry: 61.19105849175864
Max number of charactors in poetry: 255

The sentences 100 to 110:
自古逢秋悲寂寥，我言秋日胜春朝。晴空一鹤排云上，便引诗情到碧宵。
男儿何不带吴钩，收取关山五十州。请君暂上凌烟阁，若个书生万户侯。
早梅发高树，回映楚天碧。朔吹飘夜香，繁霜滋晓白。欲为万里赠，杳杳山水隔。寒英坐销落，何用慰远客？
孤山寺北贾亭西，水面初平云脚低。几处早莺争暖树，谁家新燕啄春泥。乱花渐欲迷人眼，浅草才能没马蹄。最爱湖东行不足，绿杨阴里白沙堤。
闽国扬帆后，蟾蜍亏复圆。秋风吹渭水，落叶满长安。此地聚会夕，当时雷雨寒。兰桡殊未返，消息海云端。
昔看黄菊与君别，今听玄蝉我却回。五夜飕溜枕前觉，一夜颜妆镜中来。马思边草拳毛动，雕眄青云睡眼开。天地肃清堪开望，为君扶病上高台。
巴山楚水凄凉地，二十三年弃置身。怀旧空吟闻笛赋，到乡翻似烂柯人。沉舟侧畔千帆过，病树前头万木春。今日听君歌一曲，暂凭杯酒长精神。
荒村带返照，落叶乱纷纷。古路无行客，寒山独见君。野桥经雨断，涧水向田分。不为怜同病，何人到白云。
一封朝奏九重天，夕贬潮州路八千。欲为圣明除弊事，肯将衰朽惜残年。云横秦岭家何在，雪拥蓝关马不前。知汝远来应有意，好收吾骨瘴江边。
清晨入古寺，初日照高林。竹径通幽处，禅房花木深。山光悦鸟性，潭影空人心。万籁此俱寂，但余钟磬声。


### Preprocess Functions
- Lookup Table

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    vocab = set(text)
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    vocab_to_int['PAD'] = len(vocab_to_int)
    vocab_to_int['EOS'] = len(vocab_to_int)
    int_to_vocab[vocab_to_int['PAD']] = 'PAD'
    int_to_vocab[vocab_to_int['EOS']] = 'EOS'
    return vocab_to_int, int_to_vocab
tests.test_create_lookup_tables(create_lookup_tables)

Test Passed


In [4]:
vocab_to_int, int_to_vocab = create_lookup_tables(text)

## Build the Neural Network
### Check the version of Tensorflow and Access to GPU

In [5]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use Tensorflow version 1.0 or newer'
print('Tensorflow Version: {}'.format(tf.__version__))

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Tensorflow Version: 1.1.0
Default GPU Device: /gpu:0


### Get inputs

In [6]:
def get_inputs():
    input_placeholder = tf.placeholder(tf.int32, [None, None], name='input')
    target_placeholder = tf.placeholder(tf.int32, [None, None], name='target')
    learning_rate_placeholder = tf.placeholder(tf.float32, name='learning_rate')
    return input_placeholder, target_placeholder, learning_rate_placeholder

tests.test_get_inputs(get_inputs)

(?, ?)
Test Passed


### Word Embedding

In [7]:
def get_embed(input_data, vocab_size, embed_dim):
    embed = tf.contrib.layers.embed_sequence(input_data, vocab_size=vocab_size, embed_dim=embed_dim)
    return embed

tests.test_get_embed(get_embed)

Test Passed


### Build RNN Cell and Initialize

In [8]:
def get_init_cell(batch_size, rnn_size, num_layers):
    def create_cell(rnn_size):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.75)
        return lstm
    cell = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return cell, initial_state

### Build RNN

In [9]:
def build_rnn(cell, inputs):
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(state, name="final_state")
    return outputs, final_state

tests.test_build_rnn(build_rnn)

Test Passed


### Build the Neural Network

In [10]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    rnn_outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(rnn_outputs, vocab_size, activation_fn=None)
    return logits, final_state

tests.test_build_nn(build_nn)

Test Passed


### Batches
This is the most important part in this project.Both X data and Y data are from one whole poetry. For training length, I will use '<PAD>' to fill the short poetries.

In [11]:
def pad_poetry_batch(poetry_batch, pad_int, vocab_to_int):
    int_poetry_batch = [[vocab_to_int[word] for word in list(poetry)] for poetry in poetry_batch]
    max_poetry_length = max([len(poetry) for poetry in int_poetry_batch])
    return [poetry + [pad_int] * (max_poetry_length - len(poetry)) for poetry in int_poetry_batch]

# tests.test_pad_poetry_batch(pad_poetry_batch)

In [12]:
def get_validate_poetries(text):
    poetries = text.split("\n")
    validate_poetries = []
    for poetry in poetries:
        if len(poetry) > 1:
            validate_poetries.append(poetry)
    return validate_poetries

In [13]:
def get_batches(text, batch_size, vocab_to_int, pad_id, eos_id):
    poetries = get_validate_poetries(text)
    x_batches = []
    y_batches = []
    x_batches_lengths = []
    y_batches_lengths = []
    for batch_i in range(0, len(poetries) // batch_size):
        start_i = batch_i * batch_size
        poetry_batch = poetries[start_i:start_i+batch_size]
        source_batch = poetry_batch
        target_batch = [poetry[1:]+poetry[0] for poetry in source_batch]
        pad_sources_batch = pad_poetry_batch(poetry_batch, pad_id, vocab_to_int)
        pad_targets_batch = pad_poetry_batch(target_batch, pad_id, vocab_to_int)
        
        pad_sources_lengths = [len(source) for source in pad_sources_batch]
        pad_targets_lengths = [len(target) for target in pad_targets_batch]
        
        x_batches.append(np.array(pad_sources_batch))
        y_batches.append(np.array(pad_targets_batch))
        x_batches_lengths.append(pad_sources_lengths)
        y_batches_lengths.append(pad_targets_lengths)
    
    return x_batches, y_batches, np.array(x_batches_lengths), np.array(y_batches_lengths)

## Neural Network Training

### Hyperparameters

In [17]:
num_epochs = 128
batch_size = 15
rnn_size = 512
num_layers = 2
embed_dim = 512
learning_rate = 0.003
show_every_n_batches = 50

save_dir = './save_v2'

## Build the Graph

In [18]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, num_layers)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    
    probs = tf.nn.softmax(logits, name='probs')
    
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]])
    )
    
    optimizer = tf.train.AdamOptimizer(lr)
    
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    

In [19]:
x_batches, y_batches, x_batches_lengths, y_batches_lengths = get_batches(text, batch_size, 
                                                                         vocab_to_int, vocab_to_int['PAD'], 
                                                                         vocab_to_int['EOS'])
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: x_batches[0]})
        for batch_i in range(len(x_batches)):
            feed = {
                input_text: x_batches[batch_i],
                targets: y_batches[batch_i],
                initial_state: state,
                lr: learning_rate
            }
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
            if (epoch_i * len(x_batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{} train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(x_batches),
                    train_loss))
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/2739 train_loss = 8.865
Epoch   0 Batch   50/2739 train_loss = 4.824
Epoch   0 Batch  100/2739 train_loss = 4.426
Epoch   0 Batch  150/2739 train_loss = 4.471
Epoch   0 Batch  200/2739 train_loss = 3.748
Epoch   0 Batch  250/2739 train_loss = 3.043
Epoch   0 Batch  300/2739 train_loss = 3.432
Epoch   0 Batch  350/2739 train_loss = 3.538
Epoch   0 Batch  400/2739 train_loss = 2.403
Epoch   0 Batch  450/2739 train_loss = 3.943
Epoch   0 Batch  500/2739 train_loss = 4.262
Epoch   0 Batch  550/2739 train_loss = 3.410
Epoch   0 Batch  600/2739 train_loss = 1.993
Epoch   0 Batch  650/2739 train_loss = 2.777
Epoch   0 Batch  700/2739 train_loss = 3.658
Epoch   0 Batch  750/2739 train_loss = 4.233
Epoch   0 Batch  800/2739 train_loss = 2.488
Epoch   0 Batch  850/2739 train_loss = 3.012
Epoch   0 Batch  900/2739 train_loss = 3.393
Epoch   0 Batch  950/2739 train_loss = 2.316
Epoch   0 Batch 1000/2739 train_loss = 3.038
Epoch   0 Batch 1050/2739 train_loss = 5.574
Epoch   0 

Epoch   3 Batch  933/2739 train_loss = 2.156
Epoch   3 Batch  983/2739 train_loss = 4.513
Epoch   3 Batch 1033/2739 train_loss = 1.435
Epoch   3 Batch 1083/2739 train_loss = 1.709
Epoch   3 Batch 1133/2739 train_loss = 3.589
Epoch   3 Batch 1183/2739 train_loss = 2.487
Epoch   3 Batch 1233/2739 train_loss = 1.956
Epoch   3 Batch 1283/2739 train_loss = 5.002
Epoch   3 Batch 1333/2739 train_loss = 4.647
Epoch   3 Batch 1383/2739 train_loss = 2.672
Epoch   3 Batch 1433/2739 train_loss = 2.021
Epoch   3 Batch 1483/2739 train_loss = 4.044
Epoch   3 Batch 1533/2739 train_loss = 2.474
Epoch   3 Batch 1583/2739 train_loss = 2.588
Epoch   3 Batch 1633/2739 train_loss = 1.516
Epoch   3 Batch 1683/2739 train_loss = 2.472
Epoch   3 Batch 1733/2739 train_loss = 3.790
Epoch   3 Batch 1783/2739 train_loss = 4.516
Epoch   3 Batch 1833/2739 train_loss = 3.521
Epoch   3 Batch 1883/2739 train_loss = 4.164
Epoch   3 Batch 1933/2739 train_loss = 3.557
Epoch   3 Batch 1983/2739 train_loss = 4.736
Epoch   3 

Epoch   6 Batch 1866/2739 train_loss = 1.514
Epoch   6 Batch 1916/2739 train_loss = 2.843
Epoch   6 Batch 1966/2739 train_loss = 4.832
Epoch   6 Batch 2016/2739 train_loss = 1.351
Epoch   6 Batch 2066/2739 train_loss = 4.648
Epoch   6 Batch 2116/2739 train_loss = 2.490
Epoch   6 Batch 2166/2739 train_loss = 4.024
Epoch   6 Batch 2216/2739 train_loss = 3.403
Epoch   6 Batch 2266/2739 train_loss = 4.682
Epoch   6 Batch 2316/2739 train_loss = 4.802
Epoch   6 Batch 2366/2739 train_loss = 2.485
Epoch   6 Batch 2416/2739 train_loss = 4.510
Epoch   6 Batch 2466/2739 train_loss = 1.106
Epoch   6 Batch 2516/2739 train_loss = 1.988
Epoch   6 Batch 2566/2739 train_loss = 4.448
Epoch   6 Batch 2616/2739 train_loss = 3.295
Epoch   6 Batch 2666/2739 train_loss = 3.968
Epoch   6 Batch 2716/2739 train_loss = 2.140
Epoch   7 Batch   27/2739 train_loss = 1.801
Epoch   7 Batch   77/2739 train_loss = 2.027
Epoch   7 Batch  127/2739 train_loss = 3.285
Epoch   7 Batch  177/2739 train_loss = 2.456
Epoch   7 

Epoch  10 Batch   60/2739 train_loss = 2.455
Epoch  10 Batch  110/2739 train_loss = 2.255
Epoch  10 Batch  160/2739 train_loss = 2.224
Epoch  10 Batch  210/2739 train_loss = 1.773
Epoch  10 Batch  260/2739 train_loss = 2.408
Epoch  10 Batch  310/2739 train_loss = 2.869
Epoch  10 Batch  360/2739 train_loss = 2.655
Epoch  10 Batch  410/2739 train_loss = 2.208
Epoch  10 Batch  460/2739 train_loss = 1.663
Epoch  10 Batch  510/2739 train_loss = 3.234
Epoch  10 Batch  560/2739 train_loss = 2.512
Epoch  10 Batch  610/2739 train_loss = 1.572
Epoch  10 Batch  660/2739 train_loss = 2.423
Epoch  10 Batch  710/2739 train_loss = 2.032
Epoch  10 Batch  760/2739 train_loss = 3.674
Epoch  10 Batch  810/2739 train_loss = 2.724
Epoch  10 Batch  860/2739 train_loss = 3.819
Epoch  10 Batch  910/2739 train_loss = 2.176
Epoch  10 Batch  960/2739 train_loss = 4.506
Epoch  10 Batch 1010/2739 train_loss = 1.402
Epoch  10 Batch 1060/2739 train_loss = 3.246
Epoch  10 Batch 1110/2739 train_loss = 1.680
Epoch  10 

Epoch  13 Batch  993/2739 train_loss = 3.197
Epoch  13 Batch 1043/2739 train_loss = 2.686
Epoch  13 Batch 1093/2739 train_loss = 1.621
Epoch  13 Batch 1143/2739 train_loss = 1.941
Epoch  13 Batch 1193/2739 train_loss = 1.912
Epoch  13 Batch 1243/2739 train_loss = 3.122
Epoch  13 Batch 1293/2739 train_loss = 1.457
Epoch  13 Batch 1343/2739 train_loss = 2.176
Epoch  13 Batch 1393/2739 train_loss = 2.278
Epoch  13 Batch 1443/2739 train_loss = 3.480
Epoch  13 Batch 1493/2739 train_loss = 2.607
Epoch  13 Batch 1543/2739 train_loss = 1.807
Epoch  13 Batch 1593/2739 train_loss = 2.589
Epoch  13 Batch 1643/2739 train_loss = 3.244
Epoch  13 Batch 1693/2739 train_loss = 3.800
Epoch  13 Batch 1743/2739 train_loss = 4.209
Epoch  13 Batch 1793/2739 train_loss = 2.866
Epoch  13 Batch 1843/2739 train_loss = 2.286
Epoch  13 Batch 1893/2739 train_loss = 3.564
Epoch  13 Batch 1943/2739 train_loss = 1.786
Epoch  13 Batch 1993/2739 train_loss = 1.738
Epoch  13 Batch 2043/2739 train_loss = 2.126
Epoch  13 

...

Epoch  90 Batch  540/2739 train_loss = 2.215
Epoch  90 Batch  590/2739 train_loss = 3.708
Epoch  90 Batch  640/2739 train_loss = 1.883
Epoch  90 Batch  690/2739 train_loss = 1.639
Epoch  90 Batch  740/2739 train_loss = 2.239
Epoch  90 Batch  790/2739 train_loss = 3.282
Epoch  90 Batch  840/2739 train_loss = 3.422
Epoch  90 Batch  890/2739 train_loss = 3.355
Epoch  90 Batch  940/2739 train_loss = 2.442
Epoch  90 Batch  990/2739 train_loss = 3.466
Epoch  90 Batch 1040/2739 train_loss = 2.263
Epoch  90 Batch 1090/2739 train_loss = 1.399
Epoch  90 Batch 1140/2739 train_loss = 1.289
Epoch  90 Batch 1190/2739 train_loss = 2.571
Epoch  90 Batch 1240/2739 train_loss = 1.952
Epoch  90 Batch 1290/2739 train_loss = 1.180
Epoch  90 Batch 1340/2739 train_loss = 4.037
Epoch  90 Batch 1390/2739 train_loss = 2.014
Epoch  90 Batch 1440/2739 train_loss = 1.595
Epoch  90 Batch 1490/2739 train_loss = 1.802
Epoch  90 Batch 1540/2739 train_loss = 2.078
Epoch  90 Batch 1590/2739 train_loss = 1.778
Epoch  90 

Epoch  93 Batch 1473/2739 train_loss = 2.287
Epoch  93 Batch 1523/2739 train_loss = 1.057
Epoch  93 Batch 1573/2739 train_loss = 3.910
Epoch  93 Batch 1623/2739 train_loss = 3.755
Epoch  93 Batch 1673/2739 train_loss = 1.611
Epoch  93 Batch 1723/2739 train_loss = 3.940
Epoch  93 Batch 1773/2739 train_loss = 2.495
Epoch  93 Batch 1823/2739 train_loss = 2.052
Epoch  93 Batch 1873/2739 train_loss = 3.572
Epoch  93 Batch 1923/2739 train_loss = 1.535
Epoch  93 Batch 1973/2739 train_loss = 1.762
Epoch  93 Batch 2023/2739 train_loss = 2.379
Epoch  93 Batch 2073/2739 train_loss = 3.350
Epoch  93 Batch 2123/2739 train_loss = 1.939
Epoch  93 Batch 2173/2739 train_loss = 2.999
Epoch  93 Batch 2223/2739 train_loss = 1.593
Epoch  93 Batch 2273/2739 train_loss = 2.254
Epoch  93 Batch 2323/2739 train_loss = 4.017
Epoch  93 Batch 2373/2739 train_loss = 2.658
Epoch  93 Batch 2423/2739 train_loss = 1.513
Epoch  93 Batch 2473/2739 train_loss = 2.460
Epoch  93 Batch 2523/2739 train_loss = 2.546
Epoch  93 

Epoch  96 Batch 2406/2739 train_loss = 2.149
Epoch  96 Batch 2456/2739 train_loss = 3.522
Epoch  96 Batch 2506/2739 train_loss = 4.073
Epoch  96 Batch 2556/2739 train_loss = 2.207
Epoch  96 Batch 2606/2739 train_loss = 2.506
Epoch  96 Batch 2656/2739 train_loss = 3.076
Epoch  96 Batch 2706/2739 train_loss = 2.198
Epoch  97 Batch   17/2739 train_loss = 2.231
Epoch  97 Batch   67/2739 train_loss = 4.348
Epoch  97 Batch  117/2739 train_loss = 2.897
Epoch  97 Batch  167/2739 train_loss = 2.307
Epoch  97 Batch  217/2739 train_loss = 2.460
Epoch  97 Batch  267/2739 train_loss = 2.018
Epoch  97 Batch  317/2739 train_loss = 2.527
Epoch  97 Batch  367/2739 train_loss = 2.306
Epoch  97 Batch  417/2739 train_loss = 2.109
Epoch  97 Batch  467/2739 train_loss = 2.430
Epoch  97 Batch  517/2739 train_loss = 2.293
Epoch  97 Batch  567/2739 train_loss = 2.280
Epoch  97 Batch  617/2739 train_loss = 4.006
Epoch  97 Batch  667/2739 train_loss = 2.267
Epoch  97 Batch  717/2739 train_loss = 2.171
Epoch  97 

Epoch 100 Batch  600/2739 train_loss = 1.418
Epoch 100 Batch  650/2739 train_loss = 2.049
Epoch 100 Batch  700/2739 train_loss = 2.573
Epoch 100 Batch  750/2739 train_loss = 3.181
Epoch 100 Batch  800/2739 train_loss = 1.592
Epoch 100 Batch  850/2739 train_loss = 2.194
Epoch 100 Batch  900/2739 train_loss = 2.617
Epoch 100 Batch  950/2739 train_loss = 1.487
Epoch 100 Batch 1000/2739 train_loss = 2.107
Epoch 100 Batch 1050/2739 train_loss = 3.484
Epoch 100 Batch 1100/2739 train_loss = 3.103
Epoch 100 Batch 1150/2739 train_loss = 2.384
Epoch 100 Batch 1200/2739 train_loss = 1.508
Epoch 100 Batch 1250/2739 train_loss = 1.628
Epoch 100 Batch 1300/2739 train_loss = 1.958
Epoch 100 Batch 1350/2739 train_loss = 2.129
Epoch 100 Batch 1400/2739 train_loss = 3.655
Epoch 100 Batch 1450/2739 train_loss = 1.966
Epoch 100 Batch 1500/2739 train_loss = 2.215
Epoch 100 Batch 1550/2739 train_loss = 2.591
Epoch 100 Batch 1600/2739 train_loss = 3.914
Epoch 100 Batch 1650/2739 train_loss = 2.588
Epoch 100 

KeyboardInterrupt: 